In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/product-matching-dataset/Masterfile.xlsx
/kaggle/input/product-matching-dataset/Dataset.xlsx
/kaggle/input/test-set-data/augmented_test_set.xlsx
/kaggle/input/test-data/filtered_masterfile (1).xlsx


In [19]:
# Install required libraries
!pip install sentence-transformers pandas openpyxl faiss-cpu gensim nltk fuzzywuzzy python-Levenshtein pyjarowinkler

from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import re
import Levenshtein
from fuzzywuzzy import fuzz
from pyjarowinkler.distance import get_jaro_distance
from nltk.tokenize import word_tokenize

# Arabic Normalization Function
def normalize_arabic(text):
    text = re.sub(r"[إأآا]", "ا", text)
    text = re.sub(r"[يى]", "ي", text)
    text = re.sub(r"ة", "ه", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Fuzzy Matching Functions
def levenshtein_similarity(s1, s2):
    return 1 - (Levenshtein.distance(s1, s2) / max(len(s1), len(s2)))

def jaro_winkler_similarity(s1, s2):
    return get_jaro_distance(s1, s2)

def fuzzy_wuzzy_similarity(s1, s2):
    return fuzz.token_sort_ratio(s1, s2) / 100.0

# Semantic Matching using BERT
def encode_texts(model, texts):
    return np.array(model.encode(texts, convert_to_numpy=True))

# Precompute Master Embeddings for Fast Lookups
def build_faiss_index(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index

# Function to match dataset with master file using FAISS for Fast Search
def match_dataset(master_file, dataset_file, output_file, model):
    master_df = pd.read_excel(master_file)
    dataset_df = pd.read_excel(dataset_file)
    
    master_df['Normalized Name'] = master_df['product_name_ar'].apply(normalize_arabic)
    dataset_df['Normalized Name'] = dataset_df['product_name_ar'].apply(normalize_arabic)
    
    master_embeddings = encode_texts(model, master_df['Normalized Name'].tolist())
    index = build_faiss_index(master_embeddings)
    
    dataset_embeddings = encode_texts(model, dataset_df['Normalized Name'].tolist())
    
    D, I = index.search(dataset_embeddings, 1)  # Find closest match
    
    results = []
    for i, row in dataset_df.iterrows():
        best_match_idx = I[i][0]
        best_match = master_df.iloc[best_match_idx]['product_name_ar']
        best_sku = master_df.iloc[best_match_idx]['sku']
        best_score = 1 / (1 + D[i][0])  # Convert distance to similarity
        
        results.append({
            'Item code': row['sku'],
            'product_name_ar': row['product_name_ar'],
            'Matched Item': best_match,
            'sku': best_sku,
            'Similarity Score': best_score
        })
    
    result_df = pd.DataFrame(results)
    result_df.to_excel(output_file, index=False)
    print(f"Matching completed. Output saved to {output_file}")

# Example Usage
if __name__ == "__main__":
    try:
        model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")  # Arabic-compatible BERT
    except Exception as e:
        print(f"Failed to load BERT model: {e}")
        model = None  # Fallback if model fails
    
    master_file = "/kaggle/input/product-matching-dataset/Masterfile.xlsx"
    dataset_file = "/kaggle/input/test-set-data/augmented_test_set.xlsx"
    output_file = "/kaggle/working/matched_results.xlsx"
    
    match_dataset(master_file, dataset_file, output_file, model)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Matching completed. Output saved to /kaggle/working/matched_results.xlsx


In [20]:
import openpyxl

def calculate_sku_ratio(xlsx_filepath):
    """
    Calculates the ratio of equal "Item Code" and "SKU" values to the total number of items.

    Args:
        xlsx_filepath: The path to the XLSX file.

    Returns:
        A tuple containing:
            - The ratio of equal SKUs (a float between 0 and 1).
            - A list of error messages, if any.
    """

    try:
        workbook = openpyxl.load_workbook(xlsx_filepath)
        sheet = workbook.active

        item_code_column = None
        sku_column = None
        header_row = None

        for row in sheet.iter_rows(max_row=1):  # Check only the first row for headers
            header_row = [cell.value for cell in row]
            for i, cell_value in enumerate(header_row):
                if cell_value is not None:  # Handle potential None values in headers
                    if "item code" in cell_value.lower():
                        item_code_column = i + 1
                    if "sku" in cell_value.lower():
                        sku_column = i + 1
            if item_code_column is not None and sku_column is not None:
                break

        if item_code_column is None or sku_column is None:
            return 0.0, ["'item code' or 'sku' column not found (case-insensitive) in the first row."]

        equal_sku_count = 0
        total_items = 0
        errors = []

        for row_num in range(2, sheet.max_row + 1):
            total_items += 1  # Increment total items for each row (excluding header)
            item_code_cell = sheet.cell(row=row_num, column=item_code_column)
            sku_cell = sheet.cell(row=row_num, column=sku_column)

            item_code = item_code_cell.value
            sku = sku_cell.value

            if item_code is None and sku is None:  # If both are None, consider it a match
                equal_sku_count += 1
                continue
            elif item_code is None or sku is None:  # If only one is None, it is not a match
                continue


            try:
                item_code = str(item_code)
                sku = str(sku)
            except Exception as e:
                errors.append(f"Error converting values in row {row_num}: {e}")
                continue  # Skip this row if there's a conversion error

            if item_code.strip().lower() == sku.strip().lower():
                equal_sku_count += 1

        ratio = 0.0 if total_items == 0 else equal_sku_count / total_items  # Handle division by zero
        return ratio, errors

    except FileNotFoundError:
        return 0.0, ["File not found."]
    except Exception as e:
        return 0.0, [f"An error occurred: {e}"]



# Example usage:
filepath = "/kaggle/working/matched_results.xlsx"  # Replace with your file path
ratio, errors = calculate_sku_ratio(filepath)

if errors:
    for err in errors:
        print(err)
else:
    print(f"Ratio of equal SKUs: {ratio}")

Ratio of equal SKUs: 0.8376237623762376
